# Importing Libraries

In [22]:
# Data Processing
import pandas as pd
import re
import sys
# BERT
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from scipy.spatial.distance import cosine

# Importing Data

In [23]:
df = pd.read_pickle("../data/preprocessed_dataset")

In [24]:
df.head()

,questionTitle,questionText,answerText,reflection
0,How can I keep a long distance relationship going?,"How can I keep a long distance relationship going? We weren't long distance until he joined the military. I love him and I know he loves, me but it's complicated. He said he's not going to find someone else, but I'm afraid. How do I keep our relationship going?","Hello . You are asking a very good question about how to sustain a long - distance relationship . Although maintaining a long - distance relationship has its challenges , with the proper communication , commitment , and understanding , many long - distance couples are able to thrive and maintain a close connection . Without knowing more about the complicated nature of your relationship , I wonder whether your boyfriend has given you reason not to trust him that makes afraid that he will find someone else . Has he cheated on you in the past and has shown romantic interest to another person that made you fear losing him ? Or , is your thought just a fear but not based on evidence ? Knowing that distinction is important because if it is the latter , you may benefit from refocusing on the wonderful qualities about your partner that makes you feel good about him and the relationship rather than focusing on the unknown or uncertainty of the future . The more you focus on what if situations , the more you may feel anxious about a reality that is not accurate and make you act in ways that are insecure . However , if there is reason for you to question his fidelity , you may have to speak to your boyfriend about how to build trust in the context of a long - distance relationship . To help the conversation , you may need to consider what you may need to experience or receive as support to feel safe in the relationship to build trust . Is that you wish him to contact you regularly , or to include you more in his life , or to make a clear commitment ? For many of my clients in my private practice , that may include talking to their partner often and using a variety of modalities including text , phone , and Skype . Its hard to believe in a relationship when you never talk to your partner , and its hard to build a relationship when you dont know whats going on in your partners life . Other times , it is Making sure they talk often to their long - distance partner so that they can participate in each others lives and to feel their presence . . Regular communication , understanding and caring is the key to sustaining any relationship , but this is especially true for long - distance ones . Dr . Virginia Chowwww . PsychologyResource . ca",You are asking a very good question about how to sustain a long - distance relationship
1,How can I ask my boyfriend about who he's texting?,How can I ask my boyfriend about who he's texting? We've been in a long distance relationship for two and a half years. I recently saw his phone and saw the people he texts the most and one of them was a female coworker. I don't know how to approach this situation. How do I ask him about it?.,"I agree with Sherry that in a close intimate relationship , you are entitled to ask questions about his relationship with significant others . These questions help couples to build connection and trust . Its based on the idea that if you reach out to him for whatever reason ( support , openess , understanding , empathy ) , you can count on him and can expect him to be responsive . How he responds to your question will give you an idea whether he helps you to feel more emotionally secure and builds trust or if you feel that you cannot be open with him . If your partner responds in an open and understanding manner , it usually indicates that he cares about your feelings and values your importance . If he responds in a defensive manner , it could mean that he does not like that you are questioning your trust in him or that he has something to hide . Either way , you may wish to explain that building trust is something that

In [25]:
df.shape

(2337, 4)

In [26]:
df.questionText = df.questionText.astype(str)
df.questionTitle = df.questionTitle.astype(str)

In [27]:
df = df[['questionTitle','questionText','reflection']]

In [28]:
df["context_len"] = df["questionText"].str.len()

# Loading BERT UNCASED BASE Pretrained Model

In [29]:
def initialize_tokenizer(pretrained_model):
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained(pretrained_model)
    return tokenizer

def get_index(tokenized_answer, tokenized_context):
    result=[]
    sll=len(tokenized_answer)
    for ind in (i for i,e in enumerate(tokenized_context) if e==tokenized_answer[0]):
        if tokenized_context[ind:ind+sll]==tokenized_answer:
            result = [ind,ind+sll-1]
    return result

In [30]:
pretrained_model = 'bert-base-uncased'
tokenizer = initialize_tokenizer(pretrained_model)
model = BertModel.from_pretrained(pretrained_model)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [31]:
def get_contextual_vector(model, tokenizer, question, context):
    encoded_answer_context = "[CLS] " + context + " [SEP]"
    #print (encoded_answer_context)
        # maximum words that BERT can have is 512
    encoded_answer_context = encoded_answer_context
    tokenized_encoded_answer_context = tokenizer.tokenize(encoded_answer_context)[:512]

    indexed_encoded_answer_context = tokenizer.convert_tokens_to_ids(tokenized_encoded_answer_context)
    segments_ids_encoded_answer_context= [1] * len(tokenized_encoded_answer_context)
    tokens_tensor_encoded_answer_context = torch.tensor([indexed_encoded_answer_context])
    segments_tensors_encoded_answer_context = torch.tensor([segments_ids_encoded_answer_context])

    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers_answer_context, _ = model(tokens_tensor_encoded_answer_context, segments_tensors_encoded_answer_context)
    token_vecs_answer_context= encoded_layers_answer_context[11][0]

    tokenized_answer = tokenizer.tokenize(question)
    indexes = get_index(tokenized_answer, tokenized_encoded_answer_context)

    if len(indexes) == 0:
        print ("check if question title is present in the question context")
        return None
    else:
        first_index, last_index = indexes[0], indexes[1] 
        return torch.mean(token_vecs_answer_context[first_index:last_index + 1], dim=0)
    
    

In [32]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
df.head()

,questionTitle,questionText,reflection,context_len
0,How can I keep a long distance relationship going?,"How can I keep a long distance relationship going? We weren't long distance until he joined the military. I love him and I know he loves, me but it's complicated. He said he's not going to find someone else, but I'm afraid. How do I keep our relationship going?",You are asking a very good question about how to sustain a long - distance relationship,261
1,How can I ask my boyfriend about who he's texting?,How can I ask my boyfriend about who he's texting? We've been in a long distance relationship for two and a half years. I recently saw his phone and saw the people he texts the most and one of them was a female coworker. I don't know how to approach this situation. How do I ask him about it?.,These questions help couples to build connection and trust,293
2,Why do I feel sad all the time?,Why do I feel sad all the time? I just feel sad all the time and I don't like anyone in my family. I feel like they're trying to control me and won't let me grow.,"If you feel sad on most days , it is worthwhile speaking to a psychologist to determine whether you suffer clinical depression",162
3,The Underlying Causes of Being Possessive,The Underlying Causes of Being Possessive I am extremely possessive in my relationships and this is hurting my friendships. How can I fix my underlying issues?,Its great you are able to realize there are other issues going on with someone who feels possessive,159
4,I get depressed whenever anyone I care about is too busy to be with me. What do I do?,"I get depressed whenever anyone I care about is too busy to be with me. What do I do? My last relationships have ended horribly. They just up and abandoned me. One of them I have never gotten closure with over it, leaving me emotionally wrecked. I know something's wrong with me but I don't know how to even start trying to better myself.\nI get into these depressed like states whenever anyone I care about is too busy to be with me. I know that I shouldn't act like that because people are genuinely busy, but I feel like they're doing it just because they don't want to be around me.","Frequently when working with clients , I utilize the A - B - C model developed by Albert Ellis , the founder of Rational Emotive Behavior Theory",585


In [33]:
# BERT only handles atmost 512 tokens
contextual_vectors = []
for i in range(df.shape[0]):
    vec = get_contextual_vector(model, tokenizer, df.loc[i,"questionTitle"], df.loc[i,"questionText"])
    if vec != None:
        contextual_vectors.append(vec)
    else:
        print (i)
        contextual_vectors.append([0 for i in range(768)])

check if question title is present in the question context
873
check if question title is present in the question context
880
check if question title is present in the question context
1291
check if question title is present in the question context
1327
check if question title is present in the question context
1346
check if question title is present in the question context
1429
check if question title is present in the question context
1482
check if question title is present in the question context
1595
check if question title is present in the question context
1764
check if question title is present in the question context
1772
check if question title is present in the question context
1786
check if question title is present in the question context
1798
check if question title is present in the question context
1843
check if question title is present in the question context
1897
check if question title is present in the question context
1927
check if question title is present in the 

In [34]:
df["contextual_vector"] = pd.Series(contextual_vectors)

In [35]:
df.memory_usage(deep=True, index = True).sum()

5445990

# Save the Dataframe with Contextual Vectors

In [36]:
df.to_pickle("../data/contextual_similarity_df")

# Load the Saved DataFrame

In [37]:
df = pd.read_pickle("../data/contextual_similarity_df")

# Generate Reflection

In [58]:
user_question = "Why am I feeling uncomfortable at social situations, especially parties?"

user_question_context = "I am an introvert and feel anxious in parties, for example I was at an event at USC last Friday, and after 20 min I left because I felt uncomfortable with all the new people there and I just wanted to be by myself. Why is that?"

user_question_context = user_question + user_question_context

In [59]:
user_vec = get_contextual_vector(model, tokenizer, user_question, user_question_context)

In [60]:
for i in range(df.shape[0]):
    df.loc[i, "similarity_score"] = 1 - cosine(user_vec, df.loc[i, "contextual_vector"]) 

In [61]:
df[df.similarity_score == df.similarity_score.max()]

,questionTitle,questionText,reflection,context_len,contextual_vector,similarity_score
541,Why do I feel anxious and uncomfortable at parties or big social situations?,"Why do I feel anxious and uncomfortable at parties or big social situations? Example would be homecoming? I was at my high school's home football game on Friday, and I left because I felt uncomfortable with all the people there and I just wanted to be by myself. Why is that?","Oh , that is so frustrating ! Have you ever heard of social anxiety ? Sounds like you may be suffering from it",275,"[tensor(-0.2358), tensor(0.0309), tensor(0.1955), tensor(-0.0268), tensor(0.1350), tensor(-0.1140), tensor(-0.2105), tensor(0.7592), tensor(-0.5971), tensor(-0.2915), tensor(0.0340), tensor(-0.2916), tensor(-0.3366), tensor(0.1365), tensor(-0.2390), tensor(0.4136), tensor(0.8888), tensor(0.1472), tensor(-0.2646), tensor(0.7480), tensor(0.2853), tensor(-0.0635), tensor(-0.6850), tensor(0.5600), tensor(0.8233), tensor(0.0130), tensor(0.0011), tensor(-0.3409), tensor(-0.0205), tensor(-0.3522), tensor(0.3610), tensor(-0.0676), tensor(0.0142), tensor(0.3309), tensor(-0.2346), tensor(-0.2860), tensor(-0.1711), tensor(-0.1975), tensor(0.2608), tensor(0.1552), tensor(-0.4011), tensor(-0.9845), tensor(-0.8746), tensor(0.1349), tensor(-0.3269), tensor(-0.6830), tensor(0.5841), tensor(0.1373), tensor(0.0630), tensor(0.1155), tensor(-0.2734), tensor(0.4148), tensor(-0.4005), tensor(0.2638), tensor(0.0504), tensor(0.6712), tensor(0.0762), tensor(-0.5240), tensor(-0.4193), tensor(-0.6468), tensor(0.5736), tensor(-0.0652), tensor(0.0183), tensor(-0.4654), tensor(0.3044), tensor(0.2080), tensor(0.1291), tensor(-0.1683), tensor(-0.4428), tensor(-0.0117), tensor(-0.2723), tensor(-0.0311), tensor(-0.3912), tensor(-0.2502), tensor(-0.2450), tensor(0.5224), tensor(0.0274), tensor(-0.1409), tensor(-0.2258), tensor(0.2398), tensor(-0.2816), tensor(0.4019), tensor(-0.1102), tensor(0.6867), tensor(0.2167), tensor(0.0882), tensor(-0.0101), tensor(0.1982), tensor(0.2665), tensor(-0.1812), tensor(0.4052), tensor(0.0177), tensor(0.3830), tensor(0.2501), tensor(0.1050), tensor(-0.1326), tensor(0.5665), tensor(-0.4140), tensor(-0.2932), tensor(0.5466), ...]",0.918351


In [62]:
print ("Youper:", df[df.similarity_score == df.similarity_score.max()].reset_index().reflection[0])

Youper: Oh , that is so frustrating ! Have you ever heard of social anxiety ? Sounds like you may be suffering from it
